## Initial Imports and Loading DotEnv Variables

In [1]:
import os
import re
from dotenv import load_dotenv
from pathlib import Path
import pandas as pd
pd.set_option('display.max_colwidth', 150)

# graph_functions.py has all the functions related to the graph. You can find it in this folder
from graph_functions import connect_to_database, create_node, create_relation, populate_graph

In [2]:
# Get environment variables
dotenv_path = Path('./.env')
load_dotenv(dotenv_path=dotenv_path)

uri = os.getenv('uri') # Path to graph database ex:"bolt://localhost:7687"
username = os.getenv('user_name') # graph database's username
password = os.getenv('password') # graph database's password

## Helper Functions

In [3]:
## General helper functions that don't manipulate or connect to the graph

def evaluate_importance(importance, relation_root_label):
  if importance == "Not available": relation_name = f"low_{relation_root_label}"
  elif int(importance) >= 80: relation_name = f"strong_{relation_root_label}"
  elif int(importance) <= 40: relation_name = f"low_{relation_root_label}"
  else: relation_name = f"medium_{relation_root_label}"

  return relation_name


def get_personality_traits(coded_traits:str):
  personality_traits=['Social', 'Realistic', 'Investigative', 'Enterprising', 'Conventional', 'Artistic']
  decoded_traits = []
  for letter in coded_traits:
    if letter == "S": personality_trait = personality_traits[0]
    elif letter == "R": personality_trait = personality_traits[1]
    elif letter == "I": personality_trait = personality_traits[2]
    elif letter == "E": personality_trait = personality_traits[3]
    elif letter == "C": personality_trait = personality_traits[4]
    elif letter == "A": personality_trait = personality_traits[5]

    decoded_traits.append(personality_trait)
  
  return decoded_traits

def remove_white_spaces(text):
  text = re.sub(r"[ ]", "_", text)
  return text

## Populate Graph Database

Manually populate graph using the functions found in graph_functions.py

In [ ]:
# # Load Basic_Skills
# interests_df = pd.read_csv("../Datasets/ONet/combined csvs/Interests.csv")

# # Connect to Graph Database
# driver = connect_to_database(uri=uri, username=username, password=password)

# # Populate graph with Occupations, Personality_Trait and Create relation between Occupations and Personality_Trait
# for i in range(len(interests_df)):
#   occupation = interests_df.loc[i, 'Occupation']
#   job_zone = interests_df.loc[i, 'Job Zone']
#   personality_trait = interests_df.loc[i, 'Interest']
#   coded_personality_trait = interests_df.loc[i, 'Interest Code']

#   # Create node_1
#   create_node(driver=driver, label='Occupation', properties={'title': occupation, 'main_personality_trait': personality_trait})

#   traits = get_personality_traits(coded_traits=coded_personality_trait)
#   for trait in traits:
#     # Create node_2
#     create_node(driver=driver, label='Personality_Trait', properties={'title': trait})

#     # Create Relation
#     create_relation(
#       driver=driver,
#       n_label_1='Occupation', n_identifier_1={'title': occupation},
#       n_label_2='Personality_Trait', n_identifier_2={'title': trait},
#       relation_label='requires_personality_trait', relation_properties={'job_zone': job_zone}
#       )
# driver.close()

### Prepeocess CSVs to make them compatible with the populate_graph function found in graph functions

Abilities

In [ ]:
# Load abilities dataframe
df = pd.read_csv("../Datasets/ONet/combined csvs/Abilities.csv")

# Create an empty dataframe that will be filled according to the format needed by populate_graph()
formatted_abilities = pd.DataFrame(columns=['Node_1', 'Node_2', 'Relation'])

# Fill the new dataframe
for i in range(len(df)):
  level = df.loc[i, 'Level']
  job_zone = df.loc[i, 'Job Zone']
  occupation = remove_white_spaces(df.loc[i, 'Occupation'])
  ability = remove_white_spaces(df.loc[i, 'Ability'])
  category = remove_white_spaces(df.loc[i, 'Category'])

  importance = df.loc[i, 'Importance']
  relation_label = evaluate_importance(importance=importance, relation_root_label='need_for_ability')

  formatted_abilities.loc[i, "Node_1"] = str({'label': 'Occupation', 'properties': str({'title': occupation}), 'identifier': str({'title': occupation})})
  formatted_abilities.loc[i, "Node_2"] = str({'label': category, 'properties': str({'title': ability}), 'identifier': str({'title': ability})})
  formatted_abilities.loc[i, "Relation"] = str({'label': relation_label, 'properties': str({'importance': importance, 'level': level, 'job_zone': job_zone})})

formatted_abilities.to_csv("formatted_abilities.csv")

Basic_Skills

In [ ]:
# Load Basic_Skills dataframe
df = pd.read_csv("../Datasets/ONet/combined csvs/Basic_Skills.csv")

# Create an empty dataframe that will be filled according to the format needed by populate_graph()
formatted_basic_skills = pd.DataFrame(columns=['Node_1', 'Node_2', 'Relation'])

# Fill the new dataframe
for i in range(len(df)):
  level = df.loc[i, 'Level']
  job_zone = df.loc[i, 'Job Zone']
  occupation = remove_white_spaces(df.loc[i, 'Occupation'])
  skill = remove_white_spaces(df.loc[i, 'Skill'])
  category = remove_white_spaces(df.loc[i, 'Category'])

  importance = df.loc[i, 'Importance']
  relation_label = evaluate_importance(importance=importance, relation_root_label='need_for_basic_skill')

  formatted_basic_skills.loc[i, "Node_1"] = str({'label': 'Occupation', 'properties': str({'title': occupation}), 'identifier': str({'title': occupation})})
  formatted_basic_skills.loc[i, "Node_2"] = str({'label': 'Basic_Skill', 'properties': str({'title': skill}), 'identifier': str({'title': skill})})
  formatted_basic_skills.loc[i, "Relation"] = str({'label': relation_label, 'properties': str({'importance': importance, 'level': level, 'job_zone': job_zone})})

formatted_basic_skills.to_csv("formatted_basic_skills.csv")

Cross-Functional Skills

In [ ]:
# Load Cross-Functional Skills
df = pd.read_csv("../Datasets/ONet/combined csvs/Cross-Functional Skills.csv")

# Create an empty dataframe that will be filled according to the format needed by populate_graph()
formatted_cross_functional_skills = pd.DataFrame(columns=['Node_1', 'Node_2', 'Relation'])

# Fill new dataframe
for i in range(len(df)):
  level = df.loc[i, 'Level']
  job_zone = df.loc[i, 'Job Zone']
  occupation = remove_white_spaces(df.loc[i, 'Occupation'])
  skill = remove_white_spaces(df.loc[i, 'Skill'])
  category = remove_white_spaces(df.loc[i, 'Category'])

  importance = df.loc[i, 'Importance']
  relation_label = evaluate_importance(importance=importance, relation_root_label='need_for_cross_functional_skill')

  formatted_cross_functional_skills.loc[i, "Node_1"] = str({'label': 'Occupation', 'properties': str({'title': occupation}), 'identifier': str({'title': occupation})})
  formatted_cross_functional_skills.loc[i, "Node_2"] = str({'label': category, 'properties': str({'title': skill}), 'identifier': str({'title': skill})})
  formatted_cross_functional_skills.loc[i, "Relation"] = str({'label': relation_label, 'properties': str({'importance': importance, 'level': level, 'job_zone': job_zone})})

formatted_cross_functional_skills.to_csv("formatted_cross_functional_skills.csv")

Interests

In [13]:
# Load Interests
df = pd.read_csv("../Datasets/ONet/combined csvs/Interests.csv")

# Create an empty dataframe that will be filled according to the format needed by populate_graph()
formatted_interests = pd.DataFrame(columns=['Node_1', 'Node_2', 'Relation'])

index = -1
# Fill new dataframe
for i in range(len(df)):
  job_zone = df.loc[i, 'Job Zone']
  occupation = remove_white_spaces(df.loc[i, 'Occupation'])

  coded_personality_traits = df.loc[i, 'Interest Code']
  decoded_personality_traits = get_personality_traits(coded_traits=coded_personality_traits)

  for trait in decoded_personality_traits:
    index += 1
    print(f"{index}: {occupation} -> {trait}")
    formatted_interests.loc[index, "Node_1"] = str({'label': 'Occupation', 'properties': str({'title': occupation}), 'identifier': str({'title': occupation})})
    formatted_interests.loc[index, "Node_2"] = str({'label': 'Personality_Trait', 'properties': str({'title': trait}), 'identifier': str({'title': trait})})
    # formatted_interests.loc[index, "Relation"] = str({'label': 'need_for_personality_trait', 'properties': str({'job_zone': job_zone})})
    formatted_interests.loc[index, "Relation"] = str({'label': 'need_for_personality_trait', 'properties': str({})})


formatted_interests.to_csv("formatted_interests.csv")

0: Amusement_and_Recreation_Attendants -> Conventional
1: Amusement_and_Recreation_Attendants -> Social
2: Amusement_and_Recreation_Attendants -> Realistic
3: Baristas -> Realistic
4: Baristas -> Conventional
5: Baristas -> Social
6: Dining_Room_and_Cafeteria_Attendants_and_Bartender_Helpers -> Realistic
7: Dining_Room_and_Cafeteria_Attendants_and_Bartender_Helpers -> Conventional
8: Dining_Room_and_Cafeteria_Attendants_and_Bartender_Helpers -> Social
9: Childcare_Workers -> Social
10: Childcare_Workers -> Conventional
11: Childcare_Workers -> Artistic
12: Home_Health_Aides -> Social
13: Home_Health_Aides -> Realistic
14: Home_Health_Aides -> Conventional
15: Hosts_and_Hostesses,_Restaurant,_Lounge,_and_Coffee_Shop -> Social
16: Hosts_and_Hostesses,_Restaurant,_Lounge,_and_Coffee_Shop -> Enterprising
17: Hosts_and_Hostesses,_Restaurant,_Lounge,_and_Coffee_Shop -> Conventional
18: Nannies -> Social
19: Nannies -> Artistic
20: Personal_Care_Aides -> Social
21: Personal_Care_Aides -> Conv

Knowledge

In [ ]:
# Load Knowledge
df = pd.read_csv("../Datasets/ONet/combined csvs/Knowledge.csv")

# Create an empty dataframe that will be filled according to the format needed by populate_graph()
formatted_knowledge = pd.DataFrame(columns=['Node_1', 'Node_2', 'Relation'])

# Fill new dataframe
for i in range(len(df)):
  level = df.loc[i, 'Level']
  job_zone = df.loc[i, 'Job Zone']
  occupation = remove_white_spaces(df.loc[i, 'Occupation'])
  knowledge = remove_white_spaces(df.loc[i, 'Knowledge'])
  category = remove_white_spaces(df.loc[i, 'Category'])

  importance = df.loc[i, 'Importance']
  relation_label = evaluate_importance(importance=importance, relation_root_label='need_for_knowledge_in')

  formatted_knowledge.loc[i, "Node_1"] = str({'label': 'Occupation', 'properties': str({'title': occupation}), 'identifier': str({'title': occupation})})
  formatted_knowledge.loc[i, "Node_2"]=str({'label': 'Knowledge', 'properties': str({'title': knowledge, 'category': category}), 'identifier': str({'title': knowledge})})
  formatted_knowledge.loc[i, "Relation"] = str({'label': relation_label, 'properties': str({'importance': importance, 'level': level, 'job_zone': job_zone})})

formatted_knowledge.to_csv("formatted_knowledge.csv")

### NOTE: Moving forward, all we need is a formatted csv and then we run:

driver = connect_to_database(uri=uri, username=username, password=password)

populate_graph(driver, test)   # where test is the dataset/csv file

driver.close()

In [14]:
# Connect to graph and populate it
from graph_functions import connect_to_database, create_node, create_relation, populate_graph

driver = connect_to_database(uri=uri, username=username, password=password)

populate_graph(driver, formatted_interests)

driver.close()